Connection And importing necessary library

In [5]:
import mysql.connector
import pandas as pd
from faker import Faker
from faker.providers import bank
from collections import defaultdict
import random
import datetime
from datetime import timedelta
import itertools

In [4]:
pip install faker



  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/26/21/25e4f6d5a98ae5d90d73a0648934c4577e667a771ffa0e9aacf0d9642d69/Faker-22.0.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 217.9 kB/s eta 0:00:08
    --------------------------------------- 0.0/1.7 MB 217.9 kB/s eta 0:00:08
    --------------------------------------- 0.0/1.7 MB 178.6 kB/s eta 0:00:10
   - -------------------------------------- 0.1/1.7 MB 204.8 kB/s eta 0:00:09
   - -------------------------------------- 0.1/1.7 MB 204.8 kB/s eta 0:00:09
   - -------------------------------------- 0.1/1.7 MB 208.4 kB/s eta 0:00:08
   - -------------------------------------- 0.1/1.7 MB 208.

In [2]:
pip install mysql-connector-python


  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/a7/84/b63f11124f808b6f1e3389072bc36cc907929d7574e85f94bf8f18117fe4/mysql_connector_python-8.2.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
      --------------------------------------- 10.2/527.0 kB ? eta -:--:--
      --------------------------------------- 10.2/527.0 kB ? eta -:--:--
     -- ---------------------------------- 30.7/527.0 kB 330.3 kB/s eta 0:00:02
     -- ---------------------------------- 41.0/527.0 kB 245.8 kB/s eta 0:00:02
     -- ---------------------------------- 41.0/527.0 kB 245.8 kB/s eta 0:00:02
     ---- -------------------------------- 61.4/527.0 kB 252.2 kB/s eta 0:00:02
     ----- ------------------------------- 81.9/527.0 kB 269.5 kB/s eta 0:00:02
     ------ ------------------------------ 92.2/527.0 kB 249.8 kB/s 

# Establishing the database connection->

In [6]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="mona"
)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)

In [4]:
print(mydb)

# Database(Finance)

In [5]:
mycursor = mydb.cursor()

mycursor.execute("USE finance")

# List Of Tables

In [6]:
mycursor.execute("SHOW TABLES")  # Now you can execute the next query
tables = mycursor.fetchall()
tables

[('Customer',),
 ('Customer_Satisfaction_Survey',),
 ('Expenses',),
 ('Experience',),
 ('Financial_Institution',),
 ('Investment_Account',),
 ('Investment_Product',),
 ('Loan_Account',),
 ('Loan_Details',)]

In [7]:
mycursor.execute("DESCRIBE Loan_Details")
schema = mycursor.fetchall()
schema

[('Start_Date', b'date', 'YES', '', None, ''),
 ('No_of_Years', b'int', 'YES', '', None, ''),
 ('LoanID', b'varchar(100)', 'YES', '', None, '')]

In [8]:
fake = Faker("en_IN")
fake.add_provider(bank)

In [9]:
start_date = datetime.date.today() - datetime.timedelta(days=5*365)  # Set the start date 5 years ago
end_date = datetime.date.today()  # Set the end date as today

# Financial institution table-:

In [10]:
#Financial_Institution

for _ in range(5):
    for _ in range(10):
        institution_id = fake.uuid4()[:8]  # Generate a random Institution ID
        institution_name = fake.random_element(['State Bank of India', 'HDFC Bank', 'ICICI Bank', 'Axis Bank', 'Punjab National Bank'])  # Select a random bank name from India
        location = fake.city()  # Generate a random city name
        phone_number = fake.numerify('##########')  # Generate a random 10-digit phone number
        
        # Insert the data into the Financial_Institution table
        sql = "INSERT INTO Financial_Institution (InstitutionID, InstitutionName, Location, PhoneNumber) VALUES (%s, %s, %s, %s)"
        val = (institution_id, institution_name, location, phone_number)
        
        mycursor.execute(sql, val)  # Execute the SQL statement with the provided values
        
        # Commit the changes to the database
        mydb.commit()  

# Investment Product table-:

In [11]:
# Investment_Product

product_names=[
"Savings Accounts","Checking Accounts","Certificates of Deposit","Money Market Accounts",\
"Home Loans","Personal Loans","Auto Loans","Credit Cards","Business Loans","Lines of Credit",\
"Student Loans","Investment Accounts","Retirement Accounts","Wealth Management",\
"Insurance","Foreign Exchange","Safe Deposit Boxes","Trust Services","Merchant Services",\
"Online Banking","Mobile Banking","Payment Processing","Money Transfer","Financial Planning",\
"Estate Planning","Stocks","Bonds","Mutual Funds"]
# Retrieve the Institution IDs from the Financial_Institution table
mycursor.execute("SELECT InstitutionID FROM Financial_Institution")
institution_ids = mycursor.fetchall()
# Generate and insert data for 5 years
for _ in range(5):
    for i in range(len(product_names)):
        product_id = fake.uuid4()[:8]  # Generate a random Product ID
        product_name = product_names[i]  # Select a product name from the list
        # Select a random Institution ID from the retrieved IDs
        institution_id = random.choice(institution_ids)[0]
        # Insert the data into the Investment_Product table
        sql = "INSERT INTO Investment_Product (ProductID, ProductName, InstitutionID) VALUES (%s, %s, %s)"
        val = (product_id, product_name, institution_id)
        mycursor.execute(sql, val)  # Execute the SQL statement with the provided values
        # Commit the changes to the database
        mydb.commit()
# Repeat certain Institution IDs at least 5 times
for _ in range(5):
    repeated_institution_ids = random.sample(institution_ids, k=5)
    for institution_id in repeated_institution_ids:
        product_id = fake.uuid4()[:8]  # Generate a random Product ID
        product_name = random.choice(product_names)  # Select a random product name
        # Insert the data into the Investment_Product table
        sql = "INSERT INTO Investment_Product (ProductID, ProductName, InstitutionID) VALUES (%s, %s, %s)"
        val = (product_id, product_name, institution_id[0])
        mycursor.execute(sql, val)  # Execute the SQL statement with the provided values
        # Commit the changes to the database
        mydb.commit()

# Customer Tables-:

In [12]:
#Customer

# Retrieve the Institution IDs from the Financial_Institution table
mycursor.execute("SELECT InstitutionID FROM Financial_Institution")
existing_institution_ids = [row[0] for row in mycursor.fetchall()]

# Ensure there are at least 5 unique Institution IDs
while len(existing_institution_ids) < 5:
    # Generate a new Institution ID
    new_institution_id = fake.uuid4()[:8]
    
    # Check if the new Institution ID already exists
    if new_institution_id in existing_institution_ids:
        continue

    # Generate a new Institution Name, Location, and PhoneNumber
    new_institution_name = fake.company()
    new_location = fake.city()
    new_phone_number = fake.numerify('##########')

    # Insert the new institution into the Financial_Institution table
    sql = "INSERT INTO Financial_Institution (InstitutionID, InstitutionName, Location, PhoneNumber) VALUES (%s, %s, %s, %s)"
    val = (new_institution_id, new_institution_name, new_location, new_phone_number)
    mycursor.execute(sql, val)
    mydb.commit()

    # Add the new Institution ID to the list of existing Institution IDs
    existing_institution_ids.append(new_institution_id)

# Generate and insert data for 5 years
for _ in range(5):
    for institution_id in existing_institution_ids:
        # Randomly select the number of customers for the current Institution ID (between 5 and 6)
        num_customers = random.randint(1, 10)

        # Retrieve the existing customers for the current Institution ID
        mycursor.execute("SELECT COUNT(*) FROM Customer WHERE InstitutionID = %s", (institution_id,))
        existing_customers = mycursor.fetchone()[0]

        # Check if the existing customers are already more than the desired minimum
        if existing_customers >= num_customers:
            continue

        # Generate and insert the remaining required customers
        for _ in range(num_customers - existing_customers):
            customer_id = fake.uuid4()[:8]  # Generate a random Customer ID
            customer_name = fake.name()  # Generate a random customer name
            location = fake.city()  # Generate a random city name

            # Insert the data into the Customer table
            sql = "INSERT INTO Customer (CustomerID, CustomerName, Location, InstitutionID) VALUES (%s, %s, %s, %s)"
            val = (customer_id, customer_name, location, institution_id)

            mycursor.execute(sql, val)  # Execute the SQL statement with the provided values

        # Commit the changes to the database
        mydb.commit()


# Sample Investment Account Table-:

In [13]:
# # Retrieve the existing Customer IDs from the Customer table
# mycursor.execute("SELECT CustomerID FROM Customer")
# existing_customer_ids = [row[0] for row in mycursor.fetchall()]

# # Retrieve the existing Product IDs and names from the Investment_Product table based on desired product category
# product_category = ['Savings Accounts', 'Certificates of Deposit', 'Money Market Accounts', 'Investment Accounts',
#                     'Retirement Accounts', 'Wealth Management', 'Stocks', 'Bonds', 'Mutual Funds']
# sql = "SELECT ProductID, ProductName FROM Investment_Product WHERE ProductName IN ({})".format(
#     ', '.join(['%s'] * len(product_category)))
# mycursor.execute(sql, tuple(product_category))
# existing_products = mycursor.fetchall()

# # Generate data for 5 years or until maximum 1000 records are reached
# start_date = datetime.date.today() - datetime.timedelta(days=5 * 365)  # Set the start date 5 years ago
# end_date = datetime.date.today()  # Set the end date as today

# current_date = start_date
# record_counter = 0  # Counter variable to keep track of the number of records generated



# while current_date <= end_date and record_counter < 1000:
#     for product_id, product_name in existing_products:
#         num_records = random.randint(5, 10)  # Generate a random number of records for each product

#         for _ in range(num_records):
#             if record_counter >= 1000:
#                 break  # Break the loop once the maximum limit is reached

#             while True:
#                 account_id = fake.uuid4()[:8]  # Generate a new account ID
#                 # Check if the account ID already exists in the table
#                 mycursor.execute("SELECT AccountID FROM Investment_Account WHERE AccountID = %s", (account_id,))
#                 result = mycursor.fetchone()
#                 if not result:
#                     break  # Unique account ID found, break the loop

#             customer_id = random.choice(existing_customer_ids)

#             # Set the amount range based on the product category
#             if product_name in ['Savings Accounts', 'Certificates of Deposit', 'Money Market Accounts']:
#                 amount_range = (1000, 10000)
#             elif product_name in ['Investment Accounts', 'Retirement Accounts', 'Wealth Management']:
#                 amount_range = (10000, 100000)
#             elif product_name in ['Stocks', 'Bonds', 'Mutual Funds']:
#                 amount_range = (100000, 1000000)
#             else:
#                 amount_range = (1000, 100000)

#             account_balance = random.uniform(amount_range[0], amount_range[1])
#             account_status = random.choice(['active', 'inactive'])
#             created_at = current_date.strftime('%Y-%m-%d')

#             # Insert the data into the Investment_Account table
#             sql = "INSERT INTO Investment_Account (AccountID, CustomerID, ProductID, AccountBalance, Account_Status, Created_At) VALUES (%s, %s, %s, %s, %s, %s)"
#             val = (account_id, customer_id, product_id, account_balance, account_status, created_at)
#             mycursor.execute(sql, val)
#             mydb.commit()

#             record_counter += 1

#     current_date += datetime.timedelta(days=1)


# Investment Account Table with Variations and Trend-:

In [14]:
# Retrieve the existing Customer IDs from the Customer table
mycursor.execute("SELECT CustomerID FROM Customer")
existing_customer_ids = [row[0] for row in mycursor.fetchall()]

# Retrieve the existing Product IDs and names from the Investment_Product table based on desired product category
product_category = ['Savings Accounts', 'Certificates of Deposit', 'Money Market Accounts', 'Investment Accounts',
                    'Retirement Accounts', 'Wealth Management', 'Stocks', 'Bonds', 'Mutual Funds']
sql = "SELECT ProductID, ProductName FROM Investment_Product WHERE ProductName IN ({})".format(
    ', '.join(['%s'] * len(product_category)))
mycursor.execute(sql, tuple(product_category))
existing_products = mycursor.fetchall()

# Generate data for 5 years or until maximum 1000 records are reached
start_date = datetime.date.today() - datetime.timedelta(days=5 * 365)  # Set the start date 5 years ago
end_date = datetime.date.today()  # Set the end date as today

current_date = start_date
record_counter = 0  # Counter variable to keep track of the number of records generated


economic_conditions_weight = 0.3
tax_seasons_weight = 0.2
investment_loan_activity_weight = 0.1
seasonal_financial_goals_weight = 0.2
regulatory_changes_weight = 0.2

while current_date <= end_date:# and record_counter < 1000:
    for product_id, product_name in existing_products:
        num_records = random.randint(5, 10)  # Generate a random number of records for each product

        for _ in range(num_records):
            if record_counter >= 1000:
                break  # Break the loop once the maximum limit is reached

            while True:
                account_id = fake.uuid4()[:8]  # Generate a new account ID
                # Check if the account ID already exists in the table
                mycursor.execute("SELECT AccountID FROM Investment_Account WHERE AccountID = %s", (account_id,))
                result = mycursor.fetchone()
                if not result:
                    break  # Unique account ID found, break the loop

            customer_id = random.choice(existing_customer_ids)

            # Set the amount range based on the product category
            if product_name in ['Savings Accounts', 'Certificates of Deposit', 'Money Market Accounts']:
                amount_range = (1000, 10000)
            elif product_name in ['Investment Accounts', 'Retirement Accounts', 'Wealth Management']:
                amount_range = (10000, 100000)
            elif product_name in ['Stocks', 'Bonds', 'Mutual Funds']:
                amount_range = (100000, 1000000)
            else:
                amount_range = (1000, 100000)

            # Apply variations to the amount
            base_amount = random.uniform(amount_range[0], amount_range[1])

            # Economic conditions variation
            economic_conditions_variation = base_amount * economic_conditions_weight * random.uniform(-0.1, 0.1)

            # Tax seasons variation
            if current_date.month in [1, 2, 3, 4, 5]:  # Assuming tax seasons occur from January to May
                tax_seasons_variation = base_amount * tax_seasons_weight * 0.05
            else:
                tax_seasons_variation = 0

            # Investment loan activity variation
            if product_name in ['Investment Accounts', 'Wealth Management', 'Stocks']:
                investment_loan_activity_variation = base_amount * investment_loan_activity_weight * 0.03
            else:
                investment_loan_activity_variation = 0

            # Seasonal financial goals variation
            if current_date.month in [3, 6, 9, 12]:  # Assuming seasonal financial goals occur in March, June, September, and December
                seasonal_financial_goals_variation = base_amount * seasonal_financial_goals_weight * 0.1
            else:
                seasonal_financial_goals_variation = 0

            # Regulatory changes variation
            if current_date.year >= 2023:  # Assuming regulatory changes occur from 2023 onwards
                regulatory_changes_variation = base_amount * regulatory_changes_weight * random.uniform(-0.05, 0.05)
            else:
                regulatory_changes_variation = 0

            # Calculate the final amount with variations
            amount = base_amount + economic_conditions_variation + tax_seasons_variation + \
                     investment_loan_activity_variation + seasonal_financial_goals_variation + \
                     regulatory_changes_variation

            account_status = random.choice(['active', 'inactive'])
            created_at = current_date.strftime('%Y-%m-%d')

            # Insert the data into the Investment_Account table
            sql = "INSERT INTO Investment_Account (AccountID, CustomerID, ProductID, AccountBalance, Account_Status, Created_At) VALUES (%s, %s, %s, %s, %s, %s)"
            val = (account_id, customer_id, product_id, amount, account_status, created_at)
            mycursor.execute(sql, val)
            mydb.commit()

            record_counter += 1

    current_date += datetime.timedelta(days=1)


# Loan Account Table-:

In [15]:

# Retrieve existing customer IDs from the Customer table
mycursor.execute("SELECT CustomerID FROM Customer")
existing_customer_ids = [row[0] for row in mycursor.fetchall()]

# Retrieve existing product IDs from the Investment_Product table
mycursor.execute("SELECT ProductID FROM Investment_Product")
existing_product_ids = [row[0] for row in mycursor.fetchall()]

# Retrieve existing institution IDs from the Financial_Institution table
mycursor.execute("SELECT InstitutionID FROM Financial_Institution")
existing_institution_ids = [row[0] for row in mycursor.fetchall()]

# Define the loan types
loan_types = ["Home Loans", "Personal Loans", "Auto Loans", "Credit Cards", "Business Loans", "Student Loans", "Bonds", "Stocks"]

# Define the loan amount ranges for each loan type
loan_amount_ranges = {
    "Home Loans": (100000, 1000000),
    "Personal Loans": (5000, 50000),
    "Auto Loans": (10000, 50000),
    "Credit Cards": (500, 10000),
    "Business Loans": (50000, 1000000),
    "Student Loans": (1000, 50000),
    "Bonds": (10000, 100000),
    "Stocks": (10000, 1000000)
}

# Define the interest rate ranges for each loan type
interest_rate_ranges = {
    "Home Loans": (0.03, 0.05),
    "Personal Loans": (0.05, 0.1),
    "Auto Loans": (0.04, 0.08),
    "Credit Cards": (0.1, 0.2),
    "Business Loans": (0.03, 0.07),
    "Student Loans": (0.05, 0.1),
    "Bonds": (0.02, 0.05),
    "Stocks": (0.05, 0.1)
}

# Define the weights
economic_conditions_weight = 0.3
tax_seasons_weight = 0.2
investment_loan_activity_weight = 0.1
seasonal_financial_goals_weight = 0.2
regulatory_changes_weight = 0.2

# Generate data for 5 years or until the record limit is reached
start_date = datetime.date.today() - timedelta(days=5*365)  # Set the start date 5 years ago
end_date = datetime.date.today()  # Set the end date as today
current_date = start_date
record_limit = 1000
record_counter = 0

while current_date <= end_date and record_counter < record_limit:
    for loan_type in loan_types:
        # Generate a random number of records for each loan type
        num_records = random.randint(1, 10)
        for _ in range(num_records):
            if record_counter >= record_limit:
                break
            loan_id = fake.uuid4()[:8]
            customer_id = random.choice(existing_customer_ids)
            product_id = random.choice(existing_product_ids)
            institution_id = random.choice(existing_institution_ids)
            # Get the loan amount range for the loan type
            amount_range = loan_amount_ranges.get(loan_type, (1000, 10000))
            loan_amount = random.uniform(amount_range[0], amount_range[1])
            # Get the interest rate range for the loan type
            interest_rate_range = interest_rate_ranges.get(loan_type, (0.05, 0.1))
            interest_rate = random.uniform(interest_rate_range[0], interest_rate_range[1])
            # Apply the weights to loan amount and interest rate
            loan_amount *= (1 + economic_conditions_weight + tax_seasons_weight + investment_loan_activity_weight + seasonal_financial_goals_weight + regulatory_changes_weight)
            interest_rate *= (1 + economic_conditions_weight + tax_seasons_weight + investment_loan_activity_weight + seasonal_financial_goals_weight + regulatory_changes_weight)
            loan_status = random.choice(['approved', 'pending', 'rejected'])
            # Insert the data into the Loan_Account table
            sql = "INSERT INTO Loan_Account (LoanID, CustomerID, ProductID, LoanAmount, LoanStatus, Loan_Type, Rate_of_Intrest, InstitutionID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
            val = (loan_id, customer_id, product_id, loan_amount, loan_status, loan_type, interest_rate, institution_id)
            mycursor.execute(sql, val)
            mydb.commit()
            record_counter += 1
    current_date += timedelta(days=1)


# Customer Satisfaction table-:

In [16]:
# Retrieve the existing Customer IDs from the Customer table
mycursor.execute("SELECT CustomerId FROM Customer")
existing_customer_ids = [row[0] for row in mycursor.fetchall()]
# Retrieve the existing Institution IDs from the Financial_Institution table
mycursor.execute("SELECT InstitutionID FROM Financial_Institution")
existing_institution_ids = [row[0] for row in mycursor.fetchall()]
# Generate data for 5 years or until the record limit is reached
start_date = datetime.date.today() - timedelta(days=5*365)  # Set the start date 5 years ago
end_date = datetime.date.today()  # Set the end date as today

current_date = start_date
inserted_records = 0  # Counter for inserted records
while current_date <= end_date:
    # Iterate for each financial institution
    for institution_id in existing_institution_ids:
        # Generate a random number of surveys for each institution on each day
        num_surveys = random.randint(5, 10)  # At least 5 surveys per survey ID
        for _ in range(num_surveys):
            if inserted_records >= 1000:
                break  # Break the loop once 1000 records are inserted
            
            survey_id = fake.uuid4()[:8]
            customer_id = random.choice(existing_customer_ids)
            survey_date = current_date.strftime('%Y-%m-%d')
            nps_rating = random.randint(0, 10)
            # Determine the promoter status based on the NPS rating
            if nps_rating >= 9:
                promoter_status = 'Promoter'
            elif nps_rating <= 6:
                promoter_status = 'Detractor'
            else:
                promoter_status = 'Passive'
            # Insert the data into the Customer_Satisfaction_Survey table
            sql = "INSERT INTO Customer_Satisfaction_Survey (SurveyId, CustomerId, SurveyDate, NPSRating, PromotorStatus, InstitutionID) VALUES (%s, %s, %s, %s, %s, %s)"
            val = (survey_id, customer_id, survey_date, nps_rating, promoter_status, institution_id)
            mycursor.execute(sql, val)
            mydb.commit()
            
            inserted_records += 1  # Increment the inserted records counter
            
    if inserted_records >= 1000:
        break  # Break the loop once 1000 records are inserted
    
    current_date += timedelta(days=1)

print(f"{inserted_records} records have been inserted into the Customer_Satisfaction_Survey table.")


1000 records have been inserted into the Customer_Satisfaction_Survey table.


# Espense Table-:

In [17]:
# Retrieve the existing Financial Institution IDs from the Financial_Institution table
mycursor.execute("SELECT InstitutionId FROM Financial_Institution")
existing_institution_ids = [row[0] for row in mycursor.fetchall()]
# Define expense categories and base amounts
expense_categories=[
    "Marketing and Advertising Expenses",
    "Lead Generation Expenses",
    "Sales Team Expenses",
    "Technology and Infrastructure Expenses",
    "Events and Sponsorships Expenses",
    "Search Engine Optimization Expenses",
    "Call Center and Customer Support Expenses",
    "Salaries",
    "Rent",
    "Utilities",
    "Marketing",
    "Technology",
    "Legal Fees",
    "Office Supplies"
]
base_amounts = [5000, 3000, 1000, 2000, 1500, 1000, 500]
# Generate data for 5 years or until the record limit is reached
start_date = datetime.date.today() - timedelta(days=5*365)  # Set the start date 5 years ago
end_date = datetime.date.today()  # Set the end date as today

current_date = start_date
inserted_records = 0  # Counter for inserted records
while current_date <= end_date:
    for category, base_amount in zip(expense_categories, base_amounts):
        if inserted_records >= 1000:
            break  # Break the loop once 1000 records are inserted
            
        expense_id = fake.uuid4()[:8]
        expense_date = current_date.strftime('%Y-%m-%d')
        # Apply trends and variations
        trend = random.uniform(-0.05, 0.05)  # Trend for expense variations
        variation = random.uniform(0.9, 1.1)  # Variation factor
        # Adjust variation based on specific months or quarters
        month = current_date.month
        if month in [4, 7, 10]:  # Adjust variation for specific months (e.g., Q2, Q3, Q4)
            variation *= random.uniform(1.2, 1.3)
        elif month in [1]:  # Adjust variation for January (e.g., year-end adjustments)
            variation *= random.uniform(1.5, 1.7)
        # Randomly select a financial institution ID
        institution_id = random.choice(existing_institution_ids)
        # Calculate expense amount with trends and variations
        expense_amount = round(base_amount * (1 + trend) * variation, 2)
        # Insert the data into the Expense table
        sql = "INSERT INTO Expenses (ExpenseId, InstitutionId, ExpenseCategory, ExpenseDate, ExpenseAmount) VALUES (%s, %s, %s, %s, %s)"
        val = (expense_id, institution_id, category, expense_date, expense_amount)
        mycursor.execute(sql, val)
        mydb.commit()
        
        inserted_records += 1  # Increment the inserted records counter
        
        if inserted_records >= 1000:
            break  # Break the loop once 1000 records are inserted
    
    if inserted_records >= 1000:
        break  # Break the loop once 1000 records are inserted
    
    current_date += timedelta(days=1)

print(f"{inserted_records} records have been inserted into the Expenses table.")


1000 records have been inserted into the Expenses table.


In [18]:


# # Retrieve the existing Financial Institution IDs from the Financial_Institution table
# mycursor.execute("SELECT InstitutionID FROM Financial_Institution")
# existing_institution_ids = [row[0] for row in mycursor.fetchall()]

# # Define expense categories and base amounts
# expense_categories = [
#     "Marketing and Advertising Expenses",
#     "Lead Generation Expenses",
#     "Sales Team Expenses",
#     "Technology and Infrastructure Expenses",
#     "Events and Sponsorships Expenses",
#     "Search Engine Optimization Expenses",
#     "Call Center and Customer Support Expenses",
#     "Salaries",
#     "Rent",
#     "Utilities",
#     "Marketing",
#     "Technology",
#     "Legal Fees",
#     "Office Supplies"
# ]

# # Generate data for all 5 years
# start_date = datetime.date.today() - timedelta(days=5*365)  # Set the start date 5 years ago
# end_date = datetime.date.today()  # Set the end date as today

# current_date = start_date
# #inserted_records = 0  # Counter for inserted records

# while current_date <= end_date:# and inserted_records < 1000:
#     for category in expense_categories:
#        # if inserted_records >= 1000:
#         #    break  # Break the loop once 1000 records are inserted

#         expense_id = fake.uuid4()[:8]
#         expense_date = current_date.strftime('%Y-%m-%d')

#         # Randomly select a financial institution ID
#         institution_id = random.choice(existing_institution_ids)

#         # Generate a random expense amount
#         expense_amount = random.uniform(100, 10000)

#         # Insert the data into the Expense table
#         sql = "INSERT INTO Expenses (ExpenseID, InstitutionID, ExpenseCategory, ExpenseDate, ExpenseAmount) VALUES (%s, %s, %s, %s, %s)"
#         val = (expense_id, institution_id, category, expense_date, expense_amount)
#         mycursor.execute(sql, val)
#         mydb.commit()

#         #inserted_records += 1

#     current_date += timedelta(days=1)

# print(f"{inserted_records} records have been inserted into the Expenses table.")


# Loan Details Table-:

In [19]:
# Retrieve the existing Loan IDs from the Loan_Account table
mycursor.execute("SELECT LoanId FROM Loan_Account")
existing_loan_ids = [row[0] for row in mycursor.fetchall()]
# Generate data for 5 years
# Generate data for 5 years or until the record limit is reached
start_date = datetime.date.today() - timedelta(days=5*365)  # Set the start date 5 years ago
end_date = datetime.date.today()  # Set the end date as today

current_date = start_date
inserted_rows = 0  # Counter for inserted rows
while current_date <= end_date and inserted_rows < 1000:
    for loan_id in existing_loan_ids:
        if inserted_rows >= 1000:
            break  # Break the loop if 1000 rows are inserted
        
        start_date = current_date.strftime('%Y-%m-%d')
        no_of_years = random.randint(1, 5)  # Randomly select loan duration from 1 to 5 years
        # Insert the data into the Loan_Details table
        sql = "INSERT INTO Loan_Details (Start_Date, No_of_years, LoanId) VALUES (%s, %s, %s)"
        val = (start_date, no_of_years, loan_id)
        mycursor.execute(sql, val)
        mydb.commit()
        
        inserted_rows += 1  # Increment the inserted rows counter
    
    current_date += timedelta(days=1)

print(f"{inserted_rows} rows have been inserted into the Loan_Details table.")


1000 rows have been inserted into the Loan_Details table.


# Experience Table-:

In [20]:
# Retrieve existing institution IDs from the Financial_Institution table
mycursor.execute("SELECT InstitutionID FROM Financial_Institution")
existing_institution_ids = [row[0] for row in mycursor.fetchall()]

# Define the experience categories
experience_categories = [
    "Customer Service",
    "Online Banking",
    "Mobile Banking",
    "Branch Services",
    "Loan Processing",
    "Account Opening",
    "ATM Services",
    "Investment Services",
    "Credit Card Services",
    "Fraud Prevention",
    "Financial Advice",
    "Complaint Resolution",
    "Accessibility",
    "Digital Banking Experience",
    "Security Measures"
]

# Generate data for the Experience table
record_limit = 1000
record_counter = 0
start_date = datetime.date.today() - datetime.timedelta(days=5*365)  # Set the start date 5 years ago
end_date = datetime.date.today()  # Set the end date as today
current_date = start_date

while current_date <= end_date and record_counter < record_limit:
    institution_id = random.choice(existing_institution_ids)
    experience_category = random.choice(experience_categories)
    nps_rating = random.randint(0, 10)

    # Insert the data into the Experience table
    sql = "INSERT INTO Experience (ExperienceID, InstitutionID, ExperienceCategory, NPSRating) VALUES (%s, %s, %s, %s)"
    val = (fake.uuid4()[:8], institution_id, experience_category, nps_rating)
    mycursor.execute(sql, val)
    mydb.commit()

    record_counter += 1
    current_date += datetime.timedelta(days=1)

print(f"{record_counter} records have been inserted into the Experience table.")


1000 records have been inserted into the Experience table.


In [21]:
mycursor.execute("DESCRIBE Financial_Institution")
schema = mycursor.fetchall()
schema

[('InstitutionID', b'varchar(100)', 'NO', 'PRI', None, ''),
 ('InstitutionName', b'varchar(100)', 'YES', '', None, ''),
 ('Location', b'varchar(100)', 'YES', '', None, ''),
 ('PhoneNumber', b'varchar(20)', 'YES', '', None, '')]

In [22]:
mycursor.execute("DESCRIBE Investment_Product")
schema = mycursor.fetchall()
schema

[('ProductID', b'varchar(100)', 'NO', 'PRI', None, ''),
 ('ProductName', b'varchar(100)', 'YES', '', None, ''),
 ('InstitutionID', b'varchar(100)', 'YES', '', None, '')]

In [23]:
mycursor.execute("DESCRIBE Customer")
schema = mycursor.fetchall()
schema

[('CustomerID', b'varchar(100)', 'NO', 'PRI', None, ''),
 ('CustomerName', b'varchar(100)', 'YES', '', None, ''),
 ('Location', b'varchar(100)', 'YES', '', None, ''),
 ('InstitutionID', b'varchar(100)', 'YES', '', None, '')]

In [24]:
mycursor.execute("DESCRIBE Investment_Account")
schema = mycursor.fetchall()
schema

[('AccountID', b'varchar(100)', 'YES', '', None, ''),
 ('CustomerID', b'varchar(100)', 'YES', '', None, ''),
 ('ProductID', b'varchar(100)', 'YES', '', None, ''),
 ('Account_Status', b'varchar(100)', 'YES', '', None, ''),
 ('Created_At', b'date', 'YES', '', None, ''),
 ('AccountBalance', b'decimal(10,2)', 'YES', '', None, '')]

In [25]:
mycursor.execute("DESCRIBE Loan_Account")
schema = mycursor.fetchall()
schema

[('LoanID', b'varchar(100)', 'NO', 'PRI', None, ''),
 ('CustomerID', b'varchar(100)', 'YES', '', None, ''),
 ('ProductID', b'varchar(100)', 'YES', '', None, ''),
 ('LoanAmount', b'float', 'YES', '', None, ''),
 ('LoanStatus', b'varchar(10)', 'YES', '', None, ''),
 ('Loan_Type', b'varchar(100)', 'YES', '', None, ''),
 ('Rate_of_Intrest', b'float', 'YES', '', None, ''),
 ('InstitutionID', b'varchar(100)', 'YES', '', None, '')]

In [26]:
mycursor.execute("DESCRIBE Customer_Satisfaction_Survey")
schema = mycursor.fetchall()
schema

[('SurveyID', b'varchar(100)', 'NO', 'PRI', None, ''),
 ('CustomerID', b'varchar(100)', 'YES', '', None, ''),
 ('SurveyDate', b'date', 'YES', '', None, ''),
 ('NPSRating', b'int', 'YES', '', None, ''),
 ('PromotorStatus', b'varchar(10)', 'YES', '', None, ''),
 ('InstitutionID', b'varchar(100)', 'YES', '', None, '')]

In [27]:
mycursor.execute("DESCRIBE Loan_Details")
schema = mycursor.fetchall()
schema

[('Start_Date', b'date', 'YES', '', None, ''),
 ('No_of_Years', b'int', 'YES', '', None, ''),
 ('LoanID', b'varchar(100)', 'YES', '', None, '')]

In [28]:
mycursor.execute("DESCRIBE Expenses")
schema = mycursor.fetchall()
schema

[('ExpenseID', b'varchar(100)', 'NO', 'PRI', None, ''),
 ('ExpenseDate', b'date', 'YES', '', None, ''),
 ('ExpenseAmount', b'decimal(10,2)', 'YES', '', None, ''),
 ('InstitutionID', b'varchar(100)', 'YES', '', None, ''),
 ('ExpenseCategory', b'varchar(100)', 'YES', '', None, '')]

In [29]:
mycursor.execute("DESCRIBE Experience")
schema = mycursor.fetchall()
schema

[('ExperienceID', b'varchar(100)', 'NO', 'PRI', None, ''),
 ('InstitutionID', b'varchar(100)', 'YES', '', None, ''),
 ('ExperienceCategory', b'varchar(100)', 'YES', '', None, ''),
 ('NPSRating', b'int', 'YES', '', None, '')]